# 서울시에서 아이 키우기 가장 좋은 자치구

In [11]:
import mysql.connector
import pandas as pd

In [16]:
conn = mysql.connector.connect(
    host = 'database-1.cbwuowec22t2.ap-northeast-2.rds.amazonaws.com',
    port = 3306,
    user = 'team2',
    password = "1234",
    database = 'eda_project' 
)

cur = conn.cursor(buffered=True)

### 구 정보 불러오기

In [ ]:
cur.execute('SELECT gu_name, gu_id, total_population, child_population FROM gu')
gu_info = cur.fetchall()

gu_name = []
gu_id = []
total = []
child = []
student = []

for info in enumerate(gu_info):
    gu_name.append(info[1][0])
    gu_id.append(info[1][1])
    total.append(int(round((info[1][2]),0)))
    child.append(int(round((info[1][3]),0)))
    student.append(int(info[1][2])-int(info[1][3]))


gu = pd.DataFrame({'구 이름':gu_name,
                     '구 id':gu_id,
                     '총 인구':total,
                     '유아 인구':child,
                     '학생 인구':student})
gu.head()

In [ ]:
gu_sort = gu.set_index('구 id')
gu_sort

,구 이름,총 인구,유아 인구,학생 인구
구 id,,,,
1,종로구,204215,25403,178812
2,중구,171223,15794,155429
3,용산구,239021,30679,208343
4,성동구,328018,47985,280034
5,광진구,363467,54690,308777
6,동대문구,351403,50306,301098
7,중랑구,358630,49668,308963
8,성북구,446463,80945,365519
9,강북구,283188,38676,244513


### 학교, 학원, 독서실, 도서관 정보 불러오기

#### 학교 정보

In [44]:
cur.execute('SELECT gu_id,school_type, school_count FROM school')
school_info = cur.fetchall()

gu_id = []
school_type = []
school_count = []

for info in enumerate(school_info):
    gu_id.append(info[1][0])
    school_type.append(info[1][1])
    school_count.append(int(info[1][2]))

school = pd.DataFrame({'구 id':gu_id,
                       '학교 유형':school_type,
                       '학교 수':school_count})
school_sort = pd.pivot_table(data=school,index='구 id', values='학교 수',aggfunc='sum')
school_sort

,학교 수
구 id,
1,118
2,109
3,141
4,208
5,192
6,248
7,243
8,281
9,157


#### 학원 정보

In [47]:
cur.execute('SELECT gu_id,academy_type, academy_count FROM academy')
academy_info = cur.fetchall()

gu_id = []
academy_type = []
academy_count = []

for info in enumerate(academy_info):
    gu_id.append(info[1][0])
    academy_type.append(info[1][1])
    academy_count.append(int(info[1][2]))

academy = pd.DataFrame({'구 id':gu_id,
                       '학원 유형':academy_type,
                       '학원 수':academy_count})


academy_sort = pd.pivot_table(data=academy,index='구 id', values='학원 수', aggfunc='sum')
academy_sort

,학원 수
구 id,
1,82
2,53
3,93
4,223
5,338
6,326
7,229
8,414
9,188


#### 도서관 정보

In [51]:
cur.execute("SELECT gu_id, space_count FROM learning_space where space_type = '도서관'")
library_info = cur.fetchall()

gu_id = []
library_count = []

for info in enumerate(library_info):
    gu_id.append(info[1][0])
    library_count.append(int(info[1][1]))

library = pd.DataFrame({'구 id':gu_id,
                       '도서관 수':library_count})
library_sort = library.set_index('구 id')
library_sort.sort_index()

,도서관 수
구 id,
1,41
2,38
3,20
4,10
5,13
6,18
7,8
8,25
9,11


#### 합치기

In [57]:
total = pd.concat([gu_sort, school_sort, academy_sort, library_sort], axis=1)
total.head()

,구 이름,총 인구,유아 인구,학생 인구,학교 수,학원 수,도서관 수
구 id,,,,,,,
1,종로구,204215,25403,178812,118,82,41
2,중구,171223,15794,155429,109,53,38
3,용산구,239021,30679,208343,141,93,20
4,성동구,328018,47985,280034,208,223,10
5,광진구,363467,54690,308777,192,338,13


In [84]:
per_s = []
per_a = []
per_l = []

for gu in total.index:
    per_s.append(total.loc[gu]['학생 인구']/total.loc[gu]['학교 수'])

for gu in total.index:
    per_a.append(total.loc[gu]['학생 인구']/total.loc[gu]['학원 수'])

for gu in total.index:
    per_l.append(total.loc[gu]['학생 인구']/total.loc[gu]['도서관 수'])

total['학교당 학생 수'] = per_s
total['학원당 학생 수'] = per_a
total['도서관당 학생 수'] = per_l

total.head()

,구 이름,총 인구,유아 인구,학생 인구,학교 수,학원 수,도서관 수,학교당 학생 수,학원당 학생 수,도서관당 학생 수,학교순위,학원순위,도서관순위
구 id,,,,,,,,,,,,,
1,종로구,204215,25403,178812,118,82,41,1515.355932,2180.634146,4361.268293,19.0,23.0,2.0
2,중구,171223,15794,155429,109,53,38,1425.954128,2932.622642,4090.236842,16.0,25.0,1.0
3,용산구,239021,30679,208343,141,93,20,1477.609929,2240.247312,10417.150000,17.0,24.0,4.0
4,성동구,328018,47985,280034,208,223,10,1346.317308,1255.757848,28003.400000,12.0,19.0,19.0
5,광진구,363467,54690,308777,192,338,13,1608.213542,913.541420,23752.076923,21.0,12.0,17.0


In [85]:
total['학교순위'] = total['학교당 학생 수'].rank(ascending=False)
total['학원순위'] = total['학원당 학생 수'].rank(ascending=False)
total['도서관순위'] = total['도서관당 학생 수'].rank(ascending=False)

In [86]:
total

,구 이름,총 인구,유아 인구,학생 인구,학교 수,학원 수,도서관 수,학교당 학생 수,학원당 학생 수,도서관당 학생 수,학교순위,학원순위,도서관순위
구 id,,,,,,,,,,,,,
1,종로구,204215,25403,178812,118,82,41,1515.355932,2180.634146,4361.268293,7.0,3.0,24.0
2,중구,171223,15794,155429,109,53,38,1425.954128,2932.622642,4090.236842,10.0,1.0,25.0
3,용산구,239021,30679,208343,141,93,20,1477.609929,2240.247312,10417.150000,9.0,2.0,22.0
4,성동구,328018,47985,280034,208,223,10,1346.317308,1255.757848,28003.400000,14.0,7.0,7.0
5,광진구,363467,54690,308777,192,338,13,1608.213542,913.541420,23752.076923,5.0,14.0,9.0
6,동대문구,351403,50306,301098,248,326,18,1214.104839,923.613497,16727.666667,19.0,13.0,19.0
7,중랑구,358630,49668,308963,243,229,8,1271.452675,1349.183406,38620.375000,18.0,5.0,3.0
8,성북구,446463,80945,365519,281,414,25,1300.779359,882.896135,14620.760000,16.0,15.0,21.0
9,강북구,283188,38676,244513,157,188,11,1557.407643,1300.601064,22228.454545,6.0,6.0,13.0


In [88]:
rank = total[['구 이름','학교순위','학원순위','도서관순위']]
rank

,구 이름,학교순위,학원순위,도서관순위
구 id,,,,
1,종로구,7.0,3.0,24.0
2,중구,10.0,1.0,25.0
3,용산구,9.0,2.0,22.0
4,성동구,14.0,7.0,7.0
5,광진구,5.0,14.0,9.0
6,동대문구,19.0,13.0,19.0
7,중랑구,18.0,5.0,3.0
8,성북구,16.0,15.0,21.0
9,강북구,6.0,6.0,13.0


In [98]:
rank['평균순위'] = rank[['학교순위', '학원순위', '도서관순위']].mean(axis=1).round(2)
rank

/tmp/ipykernel_13275/4105751643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rank['평균순위'] = rank[['학교순위', '학원순위', '도서관순위']].mean(axis=1).round(2)


,구 이름,학교순위,학원순위,도서관순위,평균순위
구 id,,,,,
1,종로구,7.0,3.0,24.0,11.33
2,중구,10.0,1.0,25.0,12.00
3,용산구,9.0,2.0,22.0,11.00
4,성동구,14.0,7.0,7.0,9.33
5,광진구,5.0,14.0,9.0,9.33
6,동대문구,19.0,13.0,19.0,17.00
7,중랑구,18.0,5.0,3.0,8.67
8,성북구,16.0,15.0,21.0,17.33
9,강북구,6.0,6.0,13.0,8.33


### 병원 정보 불러오기


#### 소아과 정보

In [26]:
cur.execute("SELECT hospital_id,hospital_name, hospital_address FROM hospital where hospital_name like '%소아%'")
pediatrics_info = cur.fetchall()

pediatrics_id = []
pediatrics_name = []
pediatrics_address = []

for info in enumerate(pediatrics_info):
    pediatrics_id.append(info[1][0])
    pediatrics_name.append(info[1][1])
    pediatrics_address.append(info[1][2])

pediatrics = pd.DataFrame({'소아과 id':pediatrics_id,
                       '소아과 이름':pediatrics_name,
                       '소아과 수':pediatrics_address})
pediatrics.head()

,소아과 id,소아과 이름,소아과 수
0,8762,고만성소아과의원,서울특별시 강남구 개포로 61
1,8775,삼성키즈소아청소년과의원,서울특별시 강남구 선릉로 26
2,8781,송영명소아과의원,서울특별시 강남구 선릉로10길
3,8788,연세도우리소아청소년과의원,서울특별시 강남구 삼성로 14
4,8789,연세바다소아청소년과의원,서울특별시 강남구 언주로7길


#### 전체 병원 정보

In [32]:
cur.execute("SELECT * FROM hospital")
hospital_info = cur.fetchall()

hospital_id = []
hospital_type = []
hospital_address = []

for info in enumerate(hospital_info):
    hospital_id.append(info[1][0])
    hospital_type.append(info[1][1])
    hospital_address.append(info[1][3])

hospital = pd.DataFrame({'병원 id':hospital_id,
                       '병원 유형':hospital_type,
                       '병원 수':hospital_address})
hospital.head()

,병원 id,병원 유형,병원 수
0,8750,요양병원,서울특별시 강남구 논현로2길
1,8751,의원,서울특별시 강남구 개포로 61
2,8752,의원,서울특별시 강남구 개포로 22
3,8753,의원,서울특별시 강남구 개포로 50
4,8754,의원,서울특별시 강남구 개포로 62


In [33]:
len(hospital)

1381

In [ ]:
conn.commit()

In [ ]:
conn.close()

### 1인당 공원 면적 순위

In [23]:
query = """
    SELECT g.gu_name, p.park_type, p.park_size, p.park_for_one
    FROM gu g, park p
    WHERE g.gu_id = p.gu_id
"""


df = pd.read_sql(query, conn)

df

/tmp/ipykernel_3140/1202815108.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,gu_name,park_type,park_size,park_for_one
0,강남구,공원,7794.55,14.16
1,강남구,도보생활권공원,3702.44,6.73
2,강동구,공원,3408.12,7.36
3,강동구,도보생활권공원,2127.21,4.59
4,강북구,공원,14570.45,49.73
5,강북구,도보생활권공원,2129.13,7.27
6,강서구,공원,4519.82,7.95
7,강서구,도보생활권공원,3532.42,6.21
8,관악구,공원,12484.82,25.08
9,관악구,도보생활권공원,1968.85,3.95


In [24]:
df= df[df['park_type'] == "공원"]

# 1인당 공원 면적 기준으로 순위를 매기기
df['rank'] = df['park_for_one'].rank(ascending=False, method='min')

df_desc = df.sort_values(by='rank')

df_desc[['gu_name', 'park_for_one', 'rank']]

/tmp/ipykernel_3140/1980845052.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rank'] = df['park_for_one'].rank(ascending=False, method='min')


,gu_name,park_for_one,rank
44,종로구,76.00,1.0
4,강북구,49.73,2.0
28,서초구,36.59,3.0
18,도봉구,32.80,4.0
42,은평구,30.58,5.0
16,노원구,29.31,6.0
8,관악구,25.08,7.0
46,중구,24.32,8.0
32,성북구,19.46,9.0
26,서대문구,15.93,10.0


In [25]:
park_rank_list = [(row['gu_name'], row['rank']) for _, row in df_desc.iterrows()]

In [26]:
park_rank_list

[('종로구', 1.0),
 ('강북구', 2.0),
 ('서초구', 3.0),
 ('도봉구', 4.0),
 ('은평구', 5.0),
 ('노원구', 6.0),
 ('관악구', 7.0),
 ('중구', 8.0),
 ('성북구', 9.0),
 ('서대문구', 10.0),
 ('강남구', 11.0),
 ('중랑구', 12.0),
 ('마포구', 13.0),
 ('금천구', 14.0),
 ('동작구', 15.0),
 ('성동구', 16.0),
 ('광진구', 17.0),
 ('구로구', 18.0),
 ('강서구', 19.0),
 ('용산구', 20.0),
 ('영등포구', 21.0),
 ('강동구', 22.0),
 ('송파구', 23.0),
 ('양천구', 24.0),
 ('동대문구', 25.0)]

In [32]:
def insert_rank_park(gu, rank):
    # 구 ID 가져오기
    cur.execute('SELECT gu_id FROM gu WHERE gu_name = %s', (gu,))
    gu_id = cur.fetchone()[0]


    cur.execute(
        'UPDATE rank_for_gu SET rank_park = %s WHERE gu_id = %s', 
        (rank, gu_id))
    
    conn.commit()

In [33]:
for gu, rank in park_rank_list:
    insert_rank_park(gu, rank)

### 동별로 면적 대비 월세가 비교

In [34]:
query = """
    SELECT g.gu_name, d.dong_name, r.estate_type, r.estate_size, r.estate_rent, r.estate_deposit
    FROM gu g, real_estate r, dong d
    WHERE g.gu_id = r.gu_id AND d.dong_id = r.dong_id
"""


df = pd.read_sql(query, conn)


df

/tmp/ipykernel_3140/3067331709.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,gu_name,dong_name,estate_type,estate_size,estate_rent,estate_deposit
0,강남구,개포동,월세,63.4021,156.8270,35493.80
1,강남구,개포동,전세,65.4866,0.0000,73743.30
2,강남구,논현동,월세,43.3511,112.4050,10508.60
3,강남구,논현동,전세,54.7163,0.0000,46741.10
4,강남구,대치동,월세,61.4932,132.8380,32391.80
...,...,...,...,...,...,...
759,중랑구,상봉동,전세,46.9074,0.0000,26960.70
760,중랑구,신내동,월세,47.2308,44.2891,6466.25
761,중랑구,신내동,전세,67.5466,0.0000,31843.10
762,중랑구,중화동,월세,46.3803,50.1542,5656.78


In [35]:
df= df[df['estate_type'] == "월세"]

# 면적 대비 월세 계산 (면적 대비 월세 = 월세 / 면적)
df['면적대비_월세'] = df['estate_rent'] / df['estate_size']

# 면적 대비 월세 기준으로 오름차순으로 순위 매기기
df['rank'] = df['면적대비_월세'].rank(ascending=True, method='min')

# 동별로 순위 정렬
df_desc = df.sort_values(by='rank')

df_desc[['gu_name','dong_name', '면적대비_월세', 'rank']]

/tmp/ipykernel_3140/2171584045.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['면적대비_월세'] = df['estate_rent'] / df['estate_size']
/tmp/ipykernel_3140/2171584045.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rank'] = df['면적대비_월세'].rank(ascending=True, method='min')


,gu_name,dong_name,면적대비_월세,rank
516,용산구,용산동3가,0.365789,1.0
110,구로구,천왕동,0.551027,2.0
685,중구,남산동1가,0.572316,3.0
749,중구,회현동3가,0.601413,4.0
399,송파구,마천동,0.694995,5.0
...,...,...,...,...
547,용산구,한남동,4.319329,386.0
635,종로구,운니동,4.331683,387.0
727,중구,정동,4.382566,388.0
582,종로구,관수동,4.710595,389.0


In [36]:
rent_rank_list = [(row['gu_name'], row['dong_name'], row['rank']) for _, row in df_desc.iterrows()]

In [37]:
rent_rank_list

[('용산구', '용산동3가', 1.0),
 ('구로구', '천왕동', 2.0),
 ('중구', '남산동1가', 3.0),
 ('중구', '회현동3가', 4.0),
 ('송파구', '마천동', 5.0),
 ('구로구', '궁동', 6.0),
 ('종로구', '안국동', 7.0),
 ('구로구', '항동', 8.0),
 ('종로구', '구기동', 9.0),
 ('강북구', '우이동', 10.0),
 ('강동구', '강일동', 11.0),
 ('종로구', '창성동', 12.0),
 ('영등포구', '문래동1가', 13.0),
 ('중랑구', '신내동', 14.0),
 ('구로구', '온수동', 15.0),
 ('용산구', '청암동', 16.0),
 ('종로구', '행촌동', 17.0),
 ('강서구', '외발산동', 18.0),
 ('종로구', '평창동', 19.0),
 ('구로구', '개봉동', 20.0),
 ('구로구', '고척동', 21.0),
 ('중구', '필동2가', 22.0),
 ('용산구', '주성동', 23.0),
 ('도봉구', '쌍문동', 24.0),
 ('종로구', '누상동', 25.0),
 ('종로구', '관철동', 26.0),
 ('종로구', '부암동', 27.0),
 ('중랑구', '중화동', 28.0),
 ('종로구', '신교동', 29.0),
 ('금천구', '시흥동', 30.0),
 ('중랑구', '망우동', 31.0),
 ('송파구', '오금동', 32.0),
 ('강서구', '내발산동', 33.0),
 ('용산구', '갈월동', 34.0),
 ('중구', '남대문로5가', 35.0),
 ('은평구', '구산동', 36.0),
 ('도봉구', '도봉동', 37.0),
 ('도봉구', '방학동', 38.0),
 ('양천구', '신정동', 39.0),
 ('강서구', '개화동', 40.0),
 ('중랑구', '묵동', 41.0),
 ('은평구', '갈현동', 42.0),
 ('강남구', '율현동', 43.0),
 ('성북구', '동소

In [47]:
def insert_rank_rent(gu, dong, rank):
    # 구 ID 가져오기
    cur.execute('SELECT gu_id FROM gu WHERE gu_name = %s', (gu,))
    gu_id = cur.fetchone()[0]

    #동 ID 가져오기
    cur.execute('SELECT dong_id FROM dong WHERE dong_name = %s', (dong,))
    dong_id = cur.fetchone()[0]

    cur.execute(
            'INSERT INTO rank_for_dong (gu_id, dong_id, rank_rent) '
            'VALUES (%s, %s, %s)', 
            (gu_id, dong_id, rank))
    
    conn.commit()

In [48]:
for gu, dong, rank in rent_rank_list:
    insert_rank_rent(gu, dong, rank)